In [2]:
!pip install pyamaze

In [4]:
from pyamaze import maze, agent, COLOR
from queue import PriorityQueue
import math
import os
import time

# Define start and goal points
start_point = (15, 17)  # (row, col)
goal_point = (3, 2)     # (row, col)

In [6]:
def load_or_create_maze():
    
   # Loads a maze from 'maze.csv' if it exists.
   # Otherwise, it creates a new maze, saves it as 'maze.csv', and returns it.
    
    m = maze(15, 20)
    if os.path.exists('maze.csv'):
        print("Loading saved maze from CSV...")
        m.CreateMaze(3, 2, loadMaze="maze.csv", theme=COLOR.dark)
    else:
        print("Creating new maze and saving to CSV...")
        m.CreateMaze(3, 2, loopPercent=100, theme=COLOR.dark, saveMaze='maze.csv')
    return m

In [8]:
def BFS(m, delay=0.1):

   # Breadth-First Search (BFS) algorithm for finding the shortest path.
   # Visualises the search in real time with an agent moving step by step.
    
    start = start_point
    frontier = [start]  # Queue to hold cells to explore next (FIFO)
    explored = {start}  # Set to keep track of visited cells
    bfsPath = {}        # Dictionary to keep track of how each cell was reached

    # Set up the agent for visualization
    a = agent(m, start[0], start[1], footprints=True, color=COLOR.red, shape='square')

    while frontier:
        currCell = frontier.pop(0)  # Get the next cell from the front of the queue (FIFO)
        m.tracePath({a: [currCell]}, delay=100) # Show agent moving to the current cell
        time.sleep(delay) # Add a small delay for better visualisation

        if currCell == goal_point:
            break # stop if the goal is reached

        # Check all 4 directions (East, South, North, West)
        for d in 'ESNW':
            if m.maze_map[currCell][d]: # Check if the movement is possible
                if d == 'E':
                    childCell = (currCell[0], currCell[1] + 1)
                elif d == 'W':
                    childCell = (currCell[0], currCell[1] - 1)
                elif d == 'S':
                    childCell = (currCell[0] + 1, currCell[1])
                else:
                    childCell = (currCell[0] - 1, currCell[1])

                # Add to frontier if not explored yet
                if childCell not in explored:
                    explored.add(childCell)
                    frontier.append(childCell)
                    bfsPath[childCell] = currCell  # Record how this cell was reached

    # Reconstruct path from goal to start
    path = []
    if currCell == goal_point:
        cell = goal_point
        while cell != start:
            path.append(cell)
            cell = bfsPath[cell]
        path.reverse() # Reverse to get the path from start to goal

    return path, len(path), len(explored), explored

In [10]:
# Load or create the maze
m = load_or_create_maze()

Loading saved maze from CSV...


In [12]:
# Run BFS and get results
path_BFS, steps_BFS, _, explored = BFS(m)

# Print results
print(f"BFS Path Length: {steps_BFS}")
print(f"BFS Visited Cells: {len(explored)}")

BFS Path Length: 27
BFS Visited Cells: 296


In [13]:
# Set up a new agent to draw the final BFS path
agent_BFS = agent(m, start_point[0], start_point[1], footprints=True, color=COLOR.yellow, shape='arrow')

# Visualise the final shortest path found by BFS
m.tracePath({agent_BFS: path_BFS}, delay=100)

In [ ]:
m.run()